In [1]:
# file related
import os
from os.path import join
import datetime

# machine learning
from keras.models import Sequential
from keras.layers import Dense, Input
import tensorflow as tf
from sklearn.model_selection import train_test_split

# utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from IPython.display import Audio as play_audio
import scipy as sp
import taunet_utils
import json

# RTNeural special import
import sys
sys.path.append("../RTNeural/python/")
from model_utils import save_model

print(f"TF version {tf.__version__}")

TF version 2.12.0


In [ ]:
dataset = "AITD_Dataset_B_1"

with open(join("dataset", "saved", dataset + "in.json"), "r") as json_file:
    input_data = json.load(json_file)
with open(join("dataset", "saved", dataset + "out.json"), "r") as json_file:
    output_data = json.load(json_file)

for d in input_data:
    print(f"{d}: {input_data[d]} -> {output_data[d]}")

1 4397-spicy-drums: [0.3874, 0.0185, 0.1383, 1.0, 0.076, 0.091, 0.0953, 0.0121, 0.494, 0.0246, 0.3708, 0, 0, 0.9682] -> [0.7964052, 0.7699047666666667]
1 h_shift_48000_-1: [0.3874, 0.0791, 0.1247, 0.4809, 0.1518, 0.2294, 0.0561, 0.0134, 0.6176, 0.0277, 0.3708, 0, 0, 0.9682] -> [0.7964052, 0.7699047666666667]
1 ch_shift_48000_1: [0.3863, 0.07, 0.1261, 0.5364, 0.3471, 0.3567, 0.1374, 0.033, 0.7034, 0.0234, 0.3708, 0, 0, 0.9682] -> [0.7964052, 0.7699047666666667]
1 und_mix_0_9_None: [0.3874, 0.0253, 0.1384, 0.642, 0.183, 0.481, 0.1474, 0.0303, 0.4201, 0.3581, 0.3708, 0, 0, 0.9682] -> [0.7964052, 0.7699047666666667]
1 s_time_fade_1_in: [0.3874, 0.0224, 0.1394, 1.0, 0.1771, 0.4108, 0.1281, 0.0291, 0.4955, 0.0, 0.3708, 0, 0, 0.9682] -> [0.7964052, 0.7699047666666667]
1 _time_fade_1_out: [0.3874, 0.0182, 0.1394, 1.0, 0.181, 0.5066, 0.1014, 0.0074, 0.5036, 0.0233, 0.3708, 0, 0, 0.9682] -> [0.7964052, 0.7699047666666667]
1 time_offset_-0_5: [0.7769, 0.0191, 0.1394, 1.0, 0.0823, 0.0923, 0.1111, 

In [3]:
def create_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Dense(32, activation='relu'))
    # model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    # model.add(layers.LSTM(64, return_sequences=True))   # short term memory, useful if input data is related accross vectors
    model.add(Dense(2, kernel_regularizer=tf.keras.regularizers.l2(0.001))) # no activation (linear): continuous mapping of outputs (this is not a classification task!)
    return model

input_shape = (14,)
model = create_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                480       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 8)                 136       
                                                                 
 dense_4 (Dense)             (None, 4)                 36        
                                                                 
 dense_5 (Dense)             (None, 2)                 10        
                                                                 
Total params: 2,246
Trainable params: 2,246
Non-trainabl

In [4]:
combined_data = [(input_data[key], output_data[key]) for key in input_data.keys()]

train_data, test_data = train_test_split(combined_data, test_size=0.05, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/" + now
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

class LogRMSECallback(tf.keras.callbacks.Callback):
    def __init__(self, log_dir):
        super(LogRMSECallback, self).__init__()
        self.log_dir = log_dir

    def on_epoch_end(self, epoch, logs=None):
        mse = logs.get('val_loss')
        rmse = np.sqrt(mse)
        with tf.summary.create_file_writer(self.log_dir).as_default():
            tf.summary.scalar('val_rmse', rmse, step=epoch)
        with open(join(self.log_dir, "..", "stats.md"), 'w') as f:
            s = f"# Taunet {now} ({dataset}) stats\n"
            s += f"- MSE: {mse}\n"
            s += f"- RMSE: {rmse}\n"
            f.write(s)

%load_ext tensorboard
%tensorboard --logdir ./logs/ --port 6006

history = model.fit(
    x=np.array([item[0] for item in train_data]),
    y=np.array([item[1] for item in train_data]),
    validation_data=(
        np.array([item[0] for item in val_data]),
        np.array([item[1] for item in val_data])
    ),
    epochs=80,
    # callbacks=[tensorboard_callback, early_stopping_callback],
    callbacks=[tensorboard_callback, early_stopping_callback, LogRMSECallback(join(log_dir, "rmse"))],
    # batch_size=16
)

Reusing TensorBoard on port 6006 (pid 11444), started 13 days, 7:54:46 ago. (Use '!kill 11444' to kill it.)

Epoch 1/80
20/20 [==============================] - 3s 42ms/step - loss: 0.3217 - accuracy: 0.4455 - val_loss: 0.2525 - val_accuracy: 0.5669
Epoch 2/80
20/20 [==============================] - 0s 14ms/step - loss: 0.1794 - accuracy: 0.5176 - val_loss: 0.1045 - val_accuracy: 0.5669
Epoch 3/80
20/20 [==============================] - 0s 18ms/step - loss: 0.0852 - accuracy: 0.5176 - val_loss: 0.0763 - val_accuracy: 0.5669
Epoch 4/80
20/20 [==============================] - 0s 17ms/step - loss: 0.0792 - accuracy: 0.5176 - val_loss: 0.0757 - val_accuracy: 0.5669
Epoch 5/80
20/20 [==============================] - 0s 15ms/step - loss: 0.0773 - accuracy: 0.5144 - val_loss: 0.0742 - val_accuracy: 0.5414
Epoch 6/80
20/20 [==============================] - 0s 14ms/step - loss: 0.0757 - accuracy: 0.5513 - val_loss: 0.0730 - val_accuracy: 0.5223
Epoch 7/80
20/20 [==============================] - 0s 14ms/step - loss: 0.0747 - accuracy: 0.5657 - val_loss: 0.0726 - val_accuracy: 0.5541
Epoch 8/80
20

In [5]:
train_metrics = model.evaluate(
    np.array([item[0] for item in train_data]),
    np.array([item[1] for item in train_data]),
    verbose=0
)
val_metrics = model.evaluate(
    np.array([item[0] for item in val_data]),
    np.array([item[1] for item in val_data]),
    verbose=0
)
test_metrics = model.evaluate(
    np.array([item[0] for item in test_data]), 
    np.array([item[1] for item in test_data]),
    verbose=0
)

zero_return = model.predict((([0] * 14),))

for item in test_data:
    inp = np.array([item[0]])  # Reshape input data into a batch
    prediction = model.predict(inp)
    print("Input:", item[0])
    print("Expected Output:", item[1])
    print("Predicted Output:", prediction[0])


print({f"Train {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, train_metrics)})
print({f"Validation {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, val_metrics)})
print({f"Test {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, test_metrics)})
print("RMSE for test set:", np.sqrt(test_metrics[0]))
print(f"Zero return constants: {zero_return}")
with open(join(log_dir, "stats.md"), 'a') as f:
   f.write(f"- Zero return values: {zero_return}")

# save model as .h5, .tflite and .json
model.save(join(log_dir, "taunet.pb"))
tflite_model = tf.lite.TFLiteConverter.from_saved_model(join(log_dir, "taunet.pb")).convert()
with open(join(log_dir, "taunet.tflite"), 'wb') as f:
  f.write(tflite_model)
save_model(model, join(log_dir, "taunet.json"))

1/1 [==============================] - 0s 145ms/step
Input: [0.5787, 0.0486, 0.0889, 0.5131, 0.5311, 0.2056, 0.0383, 0.0113, 0.1668, 0.0, 0, 0.5342, 0, 0.3842000000000001]
Expected Output: [0.615136, 0.13130333333333336]
Predicted Output: [0.38812852 0.42341846]
1/1 [==============================] - 0s 31ms/step
Input: [0.6035, 0.0948, 0.1893, 0.1793, 0.2599, 0.3051, 0.086, 0.0073, 0.1627, 0.441, 0, 0.19999999999999996, 0, 0.9128000000000001]
Expected Output: [0.8142178, 0.9551131]
Predicted Output: [0.64114004 1.0031997 ]
1/1 [==============================] - 0s 35ms/step
Input: [0.6793, 0.0421, 0.1622, 0.7628, 0.5817, 0.3864, 0.0514, 0.0175, 0.2437, 0.0, 0, 0.395, 0, 0.687]
Expected Output: [0.2908304, 0.7847954666666667]
Predicted Output: [0.31151158 0.78028417]
1/1 [==============================] - 0s 38ms/step
Input: [0.9973, 0.0622, 0.1388, 0.7494, 0.4855, 0.2815, 0.1229, 0.0178, 0.1777, 0.6579, 0, 0.19999999999999996, 0, 0.9128000000000001]
Expected Output: [0.8142178, 0.9551

INFO:tensorflow:Assets written to: logs/20240616-221944\taunet.pb\assets


INFO:tensorflow:Assets written to: logs/20240616-221944\taunet.pb\assets
